# InternVL3-8B: Independent Single-Turn Bank Statement Extraction

**Protocol**: Two independent single-turn prompts + Python parsing/filtering

**Key Insight**: Multi-turn conversation degrades accuracy. LLM filtering mixes up rows. Use Python for filtering!

**Model**: InternVL3-8B with bfloat16 and multi-GPU split_model() for compatibility

---

## Complete Workflow

```
Turn 0: Image + Prompt → Headers (fresh context)
        ↓ (Python pattern matching)
Turn 1: Image + Prompt → Full Table (fresh context, dynamic markdown example)
        ↓ (Python parsing + filtering)
Schema Fields: TRANSACTION_DATES, LINE_ITEM_DESCRIPTIONS, TRANSACTION_AMOUNTS_PAID
```

### Pipeline Stages:
1. **Turn 0 (LLM)**: Identify column headers from image
2. **Pattern Matching (Python)**: Map headers to concepts (Date, Description, Debit, Credit)
3. **Turn 1 (LLM)**: Extract full markdown table using **dynamic example** matching detected column structure
4. **Python Parsing**: Parse markdown → Filter for debits → Extract schema fields

### Critical Features:
- ❌ **No Turn 2** - LLM filtering mixes up rows!
- ✅ **Python filtering** - Reliable debit/credit separation
- ✅ **Dynamic examples** - Adapt to 3/4/5 column formats
- ✅ **Markdown teaching** - InternVL3 understands markdown format for alignment
- ✅ **Tax accuracy** - Correct Debit/Credit separation critical for identifying purchases

### Why This Works:
- **Turn 0**: Clean context → accurate header identification
- **Turn 1**: Clean context + dynamic example → accurate table extraction
- **Python**: Reliable filtering for debit transactions (what taxpayer PAID)

### Model: InternVL3-8B
- **bfloat16 precision** with official multi-GPU split_model() pattern
- Higher capacity than 2B variant
- Strong vision-language capabilities
- Simple API with `model.chat()` method
- Multi-GPU: First and last LLM layers anchored to GPU 0

In [ ]:
# Cell 1: Imports and Configuration
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# Add parent directory to path AND change working directory for config file resolution
import sys
from pathlib import Path

# Get project root (parent of bank_statement/)
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Change working directory to project root so config/field_definitions.yaml is found
os.chdir(project_root)
print(f"📁 Working directory: {os.getcwd()}")

import random
import math

import numpy as np
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# IPython display for rendering markdown
from IPython.display import display, Markdown

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    # Model path - update for your environment
    "MODEL_PATH": "/home/jovyan/nfs_share/models/InternVL3-8B",
    
    # Generation settings
    "MAX_NEW_TOKENS": 4096,
    
    # Image processing - V100 optimized
    "MAX_TILES": 14,  # V100 optimized (use 18 for A10G, 36 for H200)
}

print(f"✅ Configuration loaded:")
print(f"   Model: {CONFIG['MODEL_PATH']}")
print(f"   Max tiles: {CONFIG['MAX_TILES']}")

# Set Random Seed for Reproducibility

In [2]:
# Cell 3: Set random seed

from common.reproducibility import set_seed, configure_deterministic_mode
set_seed(42)
configure_deterministic_mode(True) 

✅ Random seed set to 42 for reproducibility


# Load the model

In [3]:
# Cell 5: Load InternVL3-8B model with memory-aware loading strategy
# Supports: Multi-GPU bfloat16 OR Single-GPU 8-bit quantization

from transformers import AutoConfig, BitsAndBytesConfig

print("🔧 Loading InternVL3-8B with memory-aware strategy...")

def split_model(model_path):
    """
    Official InternVL3 multi-GPU device mapping function.
    
    Creates a custom device map that ensures the first and last layers of the
    language model stay on the same device (GPU 0) to prevent tensor placement
    mismatches during generation.
    
    From: https://internvl.readthedocs.io/en/latest/internvl3.0/quick_start.html
    """
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    
    # Since the first GPU will be used for ViT, treat it as half a GPU
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for _j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    
    # Critical components must stay on GPU 0
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    
    # CRITICAL: Force last layer back to GPU 0 to prevent device mismatch
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0
    
    return device_map

def get_gpu_free_memory(gpu_id=0):
    """Get free GPU memory in GB."""
    if not torch.cuda.is_available():
        return 0
    free_mem = torch.cuda.get_device_properties(gpu_id).total_memory - torch.cuda.memory_allocated(gpu_id)
    return free_mem / 1e9

# Check available GPU memory
world_size = torch.cuda.device_count()
print(f"🖥️  Detected {world_size} GPU(s)")

gpu0_free = get_gpu_free_memory(0)
gpu1_free = get_gpu_free_memory(1) if world_size > 1 else 0
total_free = gpu0_free + gpu1_free

print(f"💾 GPU 0 free memory: {gpu0_free:.1f} GB")
if world_size > 1:
    print(f"💾 GPU 1 free memory: {gpu1_free:.1f} GB")
print(f"💾 Total free memory: {total_free:.1f} GB")

# Decision: Use bfloat16 multi-GPU if enough memory, otherwise 8-bit quantization
# InternVL3-8B needs ~16GB in bfloat16, ~8GB in 8-bit
BFLOAT16_REQUIRED = 16.0
QUANTIZED_REQUIRED = 10.0

if total_free >= BFLOAT16_REQUIRED and world_size > 1:
    # Multi-GPU bfloat16 mode
    print(f"✅ Sufficient memory for bfloat16 multi-GPU mode")
    print("📥 Loading model with official multi-GPU split_model() pattern...")
    device_map = split_model(CONFIG['MODEL_PATH'])
    print(f"🔄 Custom device map created with {len(device_map)} entries")
    
    model = AutoModel.from_pretrained(
        CONFIG['MODEL_PATH'],
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        use_flash_attn=False,  # V100 compatible
        trust_remote_code=True,
        device_map=device_map
    ).eval()
    
    MODEL_DTYPE = torch.bfloat16
    print("✅ Loaded in bfloat16 with multi-GPU split")

elif total_free >= QUANTIZED_REQUIRED:
    # Single-GPU 8-bit quantization mode (lower memory)
    print(f"⚠️  Limited memory - using 8-bit quantization on single GPU")
    
    # Find GPU with most free memory
    if world_size > 1 and gpu1_free > gpu0_free:
        target_gpu = 1
        print(f"📥 Loading on GPU 1 (more free memory: {gpu1_free:.1f} GB)")
    else:
        target_gpu = 0
        print(f"📥 Loading on GPU 0 ({gpu0_free:.1f} GB free)")
    
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=False  # Keep on GPU
    )
    
    model = AutoModel.from_pretrained(
        CONFIG['MODEL_PATH'],
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        use_flash_attn=False,
        trust_remote_code=True,
        quantization_config=quantization_config,
        device_map={"": target_gpu}
    ).eval()
    
    MODEL_DTYPE = torch.float16
    print(f"✅ Loaded in 8-bit quantization on GPU {target_gpu}")

else:
    # Not enough memory - fail with helpful message
    raise RuntimeError(
        f"❌ Insufficient GPU memory!\n"
        f"   Available: {total_free:.1f} GB\n"
        f"   Required: {QUANTIZED_REQUIRED:.1f} GB (8-bit) or {BFLOAT16_REQUIRED:.1f} GB (bfloat16)\n"
        f"   Please free GPU memory by stopping other processes."
    )

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    CONFIG['MODEL_PATH'],
    trust_remote_code=True,
    use_fast=False
)

# Set generation config on model
model.config.max_new_tokens = CONFIG['MAX_NEW_TOKENS']

# Fix pad_token_id to suppress warnings
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Display model information
print("✅ Model and tokenizer loaded successfully!")

# GPU memory check
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    
    if device_count > 1 and hasattr(model, 'hf_device_map'):
        # Multi-GPU detailed breakdown
        print(f"🔄 Multi-GPU Distribution Analysis ({device_count} GPUs):")
        
        total_allocated = 0
        for gpu_id in range(device_count):
            gpu_allocated = torch.cuda.memory_allocated(gpu_id) / 1e9
            gpu_capacity = torch.cuda.get_device_properties(gpu_id).total_memory / 1e9
            gpu_name = torch.cuda.get_device_name(gpu_id)
            
            total_allocated += gpu_allocated
            usage_pct = (gpu_allocated / gpu_capacity) * 100 if gpu_capacity > 0 else 0
            print(f"   GPU {gpu_id} ({gpu_name}): {gpu_allocated:.1f}GB/{gpu_capacity:.0f}GB ({usage_pct:.1f}%)")
        
        from collections import Counter
        device_distribution = Counter(model.hf_device_map.values())
        print(f"✅ Model distributed: {dict(device_distribution)}")
    else:
        # Single GPU
        device = next(model.parameters()).device
        gpu_id = device.index if device.index is not None else 0
        allocated = torch.cuda.memory_allocated(gpu_id) / 1e9
        total = torch.cuda.get_device_properties(gpu_id).total_memory / 1e9
        gpu_name = torch.cuda.get_device_name(gpu_id)
        usage_pct = (allocated / total) * 100 if total > 0 else 0
        print(f"📊 GPU {gpu_id} ({gpu_name}): {allocated:.1f}GB/{total:.0f}GB ({usage_pct:.1f}%)")

# Model parameters
param_count = sum(p.numel() for p in model.parameters())
print(f"🔢 Model parameters: {param_count:,}")
print(f"🎯 Data type: {MODEL_DTYPE}")
print(f"🖥️  Device: {next(model.parameters()).device}")
print(f"🔲 Max Tiles: {CONFIG['MAX_TILES']}")

🔧 Loading InternVL3-8B with memory-aware strategy...
🖥️  Detected 2 GPU(s)
💾 GPU 0 free memory: 47.8 GB
💾 GPU 1 free memory: 47.8 GB
💾 Total free memory: 95.6 GB
✅ Sufficient memory for bfloat16 multi-GPU mode
📥 Loading model with official multi-GPU split_model() pattern...


NameError: name 'CONFIG' is not defined

# Load the image

In [ ]:
# Cell 7: Load bank statement image
# Update this path to your test image
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_amount_balance.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_date_grouped_cont.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_debit_credit.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/cba_highligted.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/low_contrast_fixed.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/nab_classic_highligted.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/westpac_debit_credit.png"
# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/synthetic_bank_images/transaction_summary.png"
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/cba_date_grouped.png"


# imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/images/image_009.png"
print("📁 Loading image...")
image = Image.open(imageName)

# CRITICAL: Store as list for multi-turn compatibility
images = [image]

print(f"✅ Image loaded: {image.size}")
print(f"✅ Images list created with {len(images)} image(s)")

In [ ]:
# Display the loaded image for visual verification
print("🖼️  Bank statement image:")
display(image)

In [ ]:
# Cell 9: Image preprocessing for InternVL3 (Official implementation)

# Official InternVL3 image preprocessing (from docs)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.229)

def build_transform(input_size):
    """Build image transformation pipeline with ImageNet normalization."""
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    """Find the closest aspect ratio from target ratios based on image dimensions."""
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=None, image_size=448, use_thumbnail=False):
    """
    Dynamically preprocess image by splitting into tiles based on aspect ratio.
    
    Args:
        image: PIL Image
        min_num: Minimum number of tiles
        max_num: Maximum number of tiles (from CONFIG['MAX_TILES'])
        image_size: Size of each tile (448 for InternVL3)
        use_thumbnail: Whether to include thumbnail image
    
    Returns:
        List of PIL Image tiles
    """
    # Use CONFIG if max_num not specified
    if max_num is None:
        max_num = CONFIG['MAX_TILES']
    
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # Generate target aspect ratios
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # Find best aspect ratio
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # Calculate target dimensions
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # Resize and split into tiles
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    
    assert len(processed_images) == blocks
    
    # Add thumbnail if requested
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    
    return processed_images

def load_image(image_file, input_size=448, max_num=None):
    """
    Load and preprocess image for InternVL3.
    
    Args:
        image_file: Path to image or PIL Image object
        input_size: Size of each tile (448 for InternVL3)
        max_num: Max number of tiles (uses CONFIG['MAX_TILES'] if None)
    
    Returns:
        pixel_values: Preprocessed tensor ready for model.chat()
    """
    # Use CONFIG if max_num not specified
    if max_num is None:
        max_num = CONFIG['MAX_TILES']
    
    # Handle both path string and PIL Image
    if isinstance(image_file, str):
        image = Image.open(image_file).convert('RGB')
    else:
        image = image_file
    
    # Build transform and preprocess
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(img) for img in images]
    pixel_values = torch.stack(pixel_values)
    
    return pixel_values

print("✅ InternVL3 image preprocessing functions defined")
print(f"🔲 Using max_num={CONFIG['MAX_TILES']} tiles (from CONFIG)")
print(f"💡 Image preprocessing: ImageNet normalization + dynamic tiling")

# Bank Statement Extraction Protocol
- Turn 0: Identify actual table headers
- Turn 1: Extract full table using dynamic markdown example
- Python: Parse, filter, and extract schema fields

In [ ]:
# Cell 9: Turn 0 - Identify table headers (prompt)
# TURN 0: Identify Table Headers
# First, identify the actual column headers used in this specific bank statement

prompt = """
Look at the transaction table in this bank statement image.

IMPORTANT STRUCTURAL NOTE:
Some bank statements show dates as section headings with multiple transactions underneath.
If you see this structure, remember that each transaction needs its explicit date in the final output.

What are the exact column header names used in the transaction table?

List each column header exactly as it appears, in order from left to right.
Do not interpret or rename them - use the EXACT text from the image.
"""

# Create message structure for Llama
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": prompt,
            },
        ],
    }
]

print("💬 TURN 0: Identifying actual table headers")
print("🤖 Generating response with Llama-3.2-Vision...")

In [ ]:
# Cell 12: Turn 0 - Execute and parse headers (InternVL3-8B version)

print("💬 TURN 0: Identifying actual table headers")
print("🤖 Generating response with InternVL3-8B...")

# Load and preprocess image for InternVL3 (uses CONFIG['MAX_TILES'])
pixel_values = load_image(imageName, input_size=448)

# Move to correct device and dtype (MODEL_DTYPE set in cell-5 based on loading mode)
# Vision model is always on GPU 0
vision_device = 'cuda:0'
pixel_values = pixel_values.to(dtype=MODEL_DTYPE, device=vision_device)

# Generate response using chat() method
cleanedOutput = model.chat(
    tokenizer=tokenizer,
    pixel_values=pixel_values,
    question=prompt,
    generation_config={
        "max_new_tokens": CONFIG['MAX_NEW_TOKENS'],
        "do_sample": False  # Greedy decoding for consistency
    }
)

# Clean InternVL3 artifacts:
# 1. Remove image markdown placeholder (![...])
# 2. Remove markdown code fences (```markdown and ```)
lines = cleanedOutput.split("\n")
cleaned_lines = []
for line in lines:
    stripped = line.strip()
    # Skip image markdown, code fences, and empty fence markers
    if stripped.startswith("!["):
        continue
    if stripped in ["```markdown", "```", "```md"]:
        continue
    cleaned_lines.append(line)

cleanedOutput = "\n".join(cleaned_lines)

print("✅ Response generated successfully!")
print("\n" + "=" * 60)
print("TURN 0 - IDENTIFIED TABLE HEADERS:")
print("=" * 60)
print(cleanedOutput)
print("=" * 60)

# CRITICAL: Parse the identified headers for use in subsequent turns
# Extract column names from the response
header_lines = [line.strip() for line in cleanedOutput.split('\n') if line.strip()]
identified_headers = []

# Look for numbered list or bullet points
for line in header_lines:
    # Remove common list markers
    cleaned = line.lstrip('0123456789.-•* ').strip()
    
    # Strip markdown bold formatting
    cleaned = cleaned.replace('**', '').replace('__', '')
    
    # Skip section headers (lines ending with colon)
    if cleaned.endswith(':'):
        continue
    
    # Skip long sentences (likely explanatory text, not headers)
    if len(cleaned) > 40:
        continue
        
    if cleaned and len(cleaned) > 2:  # Ignore very short strings
        identified_headers.append(cleaned)

print(f"\n📋 Parsed {len(identified_headers)} column headers:")
for i, header in enumerate(identified_headers, 1):
    print(f"  {i}. '{header}'")

# Store headers for use in subsequent turns
table_headers = identified_headers

# Save the table headers
output_path = Path("ivl3_8b_table_headers.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput)

print(f"\n✅ Table headers saved to: {output_path}")
print("💡 These LITERAL header names will be used in Turn 1 prompts")

In [ ]:
# # Cell 11: Turn 0 - Execute and parse headers
# # Process the input using the CORRECT multi-turn pattern
# # Based on: https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa

# textInput = processor.apply_chat_template(
#     messageDataStructure, add_generation_prompt=True
# )

# # CRITICAL: Use named parameter 'images=' with list
# inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# # Generate response with deterministic parameters
# output = model.generate(
#     **inputs,
#     max_new_tokens=2000,
#     do_sample=False,
#     temperature=None,
#     top_p=None,
# )

# # CRITICAL: Trim input tokens from output (this is the key to clean responses!)
# generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
# cleanedOutput = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

# print("✅ Response generated successfully!")
# print("\n" + "=" * 60)
# print("TURN 0 - IDENTIFIED TABLE HEADERS:")
# print("=" * 60)
# print(cleanedOutput)
# print("=" * 60)

# # CRITICAL: Parse the identified headers for use in subsequent turns
# # Extract column names from the response
# header_lines = [line.strip() for line in cleanedOutput.split('\n') if line.strip()]
# identified_headers = []

# # Look for numbered list or bullet points
# for line in header_lines:
#     # Remove common list markers
#     cleaned = line.lstrip('0123456789.-•* ').strip()
    
#     # Strip markdown bold formatting
#     cleaned = cleaned.replace('**', '').replace('__', '')
    
#     # Skip section headers (lines ending with colon)
#     if cleaned.endswith(':'):
#         continue
    
#     # Skip long sentences (likely explanatory text, not headers)
#     if len(cleaned) > 40:
#         continue
        
#     if cleaned and len(cleaned) > 2:  # Ignore very short strings
#         identified_headers.append(cleaned)

# print(f"\n📋 Parsed {len(identified_headers)} column headers:")
# for i, header in enumerate(identified_headers, 1):
#     print(f"  {i}. '{header}'")

# # Store headers for use in subsequent turns
# table_headers = identified_headers

# # Save the table headers
# output_path = Path("llama_table_headers.txt")
# with output_path.open("w", encoding="utf-8") as text_file:
#     text_file.write(cleanedOutput)

# print(f"\n✅ Table headers saved to: {output_path}")
# print("💡 These LITERAL header names will be used in Turn 1 & 2 prompts")

## Pattern Matching: Map Generic Concepts to Actual Headers

Different bank statements use different column names. Use pattern matching to identify:
- Which header represents **Date**
- Which header represents **Description/Details**  
- Which header represents **Debit/Withdrawal**

In [ ]:
# Cell 13: Pattern Matching - Map headers to generic columns
# Pattern Matching: Map extracted headers to generic concepts
# This handles variety in bank statement column naming conventions

# Pattern keywords for each concept (in priority order)
DATE_PATTERNS = ['date', 'day', 'transaction date', 'trans date']
DESCRIPTION_PATTERNS = [
    'description', 'details', 'transaction details', 'trans details',
    'particulars', 'narrative', 'transaction', 'trans'
]
DEBIT_PATTERNS = ['debit', 'withdrawal', 'withdrawals', 'paid', 'paid out', 'spent', 'dr']
CREDIT_PATTERNS = ['credit', 'deposit', 'deposits', 'received', 'cr']
BALANCE_PATTERNS = ['balance', 'bal', 'running balance']

# NEW: Pattern for single-column transaction formats (e.g., "Amount" instead of separate Debit/Credit)
AMOUNT_PATTERNS = ['amount', 'amt', 'value', 'total']

def match_header(headers, patterns, fallback=None):
    """Match a header using pattern keywords.
    
    Matching strategy:
    1. Exact match (case-insensitive)
    2. Substring match (only for patterns with length > 2 to avoid false positives)
    """
    headers_lower = [h.lower() for h in headers]
    
    # Try exact match first
    for pattern in patterns:
        for i, header_lower in enumerate(headers_lower):
            if pattern == header_lower:
                return headers[i]
    
    # Try substring match (only for patterns longer than 2 chars)
    for pattern in patterns:
        if len(pattern) > 2:  # Avoid false positives like 'cr' matching 'description'
            for i, header_lower in enumerate(headers_lower):
                if pattern in header_lower:
                    return headers[i]
    
    return fallback

# Perform pattern matching on extracted headers
date_col = match_header(table_headers, DATE_PATTERNS, fallback=table_headers[0] if table_headers else 'Date')
desc_col = match_header(table_headers, DESCRIPTION_PATTERNS, fallback=table_headers[1] if len(table_headers) > 1 else 'Description')

# NEW: First try to match a generic "Amount" column (for 4-column formats)
amount_col = match_header(table_headers, AMOUNT_PATTERNS, fallback=None)

# Use amount_col as fallback if no separate debit/credit columns exist
# This handles formats like: Date | Description | Amount | Balance
debit_col = match_header(table_headers, DEBIT_PATTERNS, fallback=amount_col if amount_col else 'Debit')
credit_col = match_header(table_headers, CREDIT_PATTERNS, fallback=amount_col if amount_col else 'Credit')
balance_col = match_header(table_headers, BALANCE_PATTERNS, fallback='Balance')

print("=" * 60)
print("PATTERN MATCHING RESULTS:")
print("=" * 60)
print(f"📋 Extracted Headers: {table_headers}")
print(f"\n🔍 Mapped Columns:")
print(f"  Date        → '{date_col}'")
print(f"  Description → '{desc_col}'")
print(f"  Debit       → '{debit_col}'")
print(f"  Credit      → '{credit_col}'")
print(f"  Balance     → '{balance_col}'")
if amount_col:
    print(f"\n💡 Single-column format detected: '{amount_col}' used for both debit and credit")
print("=" * 60)
print("\n✅ These literal column names will be used in Turn 1 and Turn 2")
print("💡 Adjust patterns above if matching fails for your bank statement format")

### 🔑 Independent Single-Turn Pattern (NOT Multi-Turn Conversation)

**CRITICAL INSIGHT**: Multi-turn conversation accumulates context and degrades accuracy.

We use **two independent single-turn prompts**, each with fresh context:

#### Key Principles:

1. **No Conversation History**: Each turn is completely independent
2. **Fresh Image Attention**: Each turn processes the image directly
3. **No Context Accumulation**: Prevents attention dilution
4. **Headers as Parameters**: Turn 0 headers used to generate dynamic examples for Turn 1
5. **Python Filtering**: LLM filtering mixes up rows - Python is reliable

#### Message Structure for Each Turn:

Every turn uses fresh structure:
```python
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "<prompt with dynamic example>"}
        ]
    }
]
```

**No assistant responses in history. No conversation accumulation.**

#### Why This Works Better:

- **Turn 0**: Clean context → accurate header identification
- **Turn 1**: Clean context + dynamic example → accurate table extraction  
- **Python**: Reliable parsing and filtering (no row mixing!)

Each turn has **full attention** on the image, not diluted by conversation history.

In [ ]:
# Cell 15: NO conversation history (independent turns)
# 
# CRITICAL: We do NOT use conversation history in this notebook.
# Each turn is completely independent with fresh context.
#
# Why? Multi-turn conversation accumulates context and degrades accuracy:
# - Turn 0: ~50 tokens → accurate
# - Turn 1 with history: ~350 tokens → attention diluted → less accurate
# - Turn 2 with history: ~2000 tokens → attention heavily diluted → row mixing!
#
# Instead: 
# - Turn 0: Fresh context → headers
# - Turn 1: Fresh context + dynamic example → full table
# - Python: Parse and filter (no LLM confusion!)

print("✅ Independent turn approach - NO conversation history")
print("💡 Each turn has fresh context with direct image access")
print("🐍 Python handles all filtering - no LLM row mixing!")

## Generate Column Aware Extraction Prompt

In [ ]:
# Cell 17: Generate Column Aware Extraction Prompt

# Build the header string using LITERAL names from Turn 0
header_string = " | ".join(table_headers)

# Build separator row with proper alignment indicators
# Date and Description columns: left-aligned (:---)
# Debit, Credit, Balance columns: right-aligned (---:)
separator_parts = []
for h in table_headers:
    h_lower = h.lower()
    # Right-align numeric columns
    if any(keyword in h_lower for keyword in ['debit', 'credit', 'balance', 'amount', 'total']):
        separator_parts.append('---:')
    else:
        # Left-align text columns (Date, Transaction, Description, etc.)
        separator_parts.append(':---')

separator_row = " | ".join(separator_parts)

# Build dynamic example rows based on detected column structure
# CRITICAL: Examples must emphasize correct Debit/Credit column alignment!
def build_dynamic_example(headers, date_col, desc_col, debit_col, credit_col, balance_col):
    """Generate example rows matching detected column structure.
    
    Emphasizes correct Debit/Credit alignment for tax purposes:
    - Debits (purchases/withdrawals) = money OUT = what taxpayer PAID
    - Credits (deposits/income) = money IN = NOT purchases
    """
    
    # Check if we have separate debit/credit columns
    has_separate_debit_credit = (debit_col in headers and credit_col in headers and debit_col != credit_col)
    
    rows = []
    
    if has_separate_debit_credit:
        # 5-column format: Emphasize Debit vs Credit with clear examples
        
        # Example 1: DEBIT transaction (purchase/withdrawal) - money OUT
        row1 = []
        for h in headers:
            if h == date_col:
                row1.append("15 Jan")
            elif h == desc_col:
                row1.append("ATM Withdrawal City Branch")
            elif h == debit_col:
                row1.append("200.00")  # Amount in DEBIT column
            elif h == credit_col:
                row1.append("")  # Credit column EMPTY
            elif h == balance_col:
                row1.append("$1,500.00 CR")
            else:
                row1.append("")
        rows.append(" | ".join(row1))
        
        # Example 2: CREDIT transaction (deposit) - money IN
        row2 = []
        for h in headers:
            if h == date_col:
                row2.append("16 Jan")
            elif h == desc_col:
                row2.append("Salary Employer Name Ref 12345")
            elif h == debit_col:
                row2.append("")  # Debit column EMPTY
            elif h == credit_col:
                row2.append("3,500.00")  # Amount in CREDIT column
            elif h == balance_col:
                row2.append("$5,000.00 CR")
            else:
                row2.append("")
        rows.append(" | ".join(row2))
        
        # Example 3: Another DEBIT transaction (purchase)
        row3 = []
        for h in headers:
            if h == date_col:
                row3.append("17 Jan")
            elif h == desc_col:
                row3.append("Online Purchase Store Name")
            elif h == debit_col:
                row3.append("150.00")  # Amount in DEBIT column
            elif h == credit_col:
                row3.append("")  # Credit column EMPTY
            elif h == balance_col:
                row3.append("$4,850.00 CR")
            else:
                row3.append("")
        rows.append(" | ".join(row3))
        
    else:
        # 4-column format: Single Amount column
        row1 = []
        for h in headers:
            if h == date_col:
                row1.append("15 Jan")
            elif h == desc_col:
                row1.append("ATM Withdrawal City Branch")
            elif h == debit_col:  # This is the Amount column
                row1.append("200.00")
            elif h == balance_col:
                row1.append("$1,500.00 CR")
            else:
                row1.append("")
        rows.append(" | ".join(row1))
        
        row2 = []
        for h in headers:
            if h == date_col:
                row2.append("16 Jan")
            elif h == desc_col:
                row2.append("Salary Employer Name Ref 12345")
            elif h == debit_col:  # This is the Amount column
                row2.append("3,500.00")
            elif h == balance_col:
                row2.append("$5,000.00 CR")
            else:
                row2.append("")
        rows.append(" | ".join(row2))
    
    return rows

def build_multiline_rule(headers):
    """Generate multi-line extraction rule using ACTUAL column structure from Turn 0."""
    num_cols = len(headers)
    
    # Find actual Debit, Credit, and Balance columns by name
    debit_idx = None
    credit_idx = None
    balance_idx = None
    
    for i, header in enumerate(headers):
        h_lower = header.lower()
        if any(p in h_lower for p in ["debit", "withdrawal", "paid", "spent", "dr"]):
            debit_idx = i
        if any(p in h_lower for p in ["credit", "deposit", "received", "cr"]):
            credit_idx = i
        if any(p in h_lower for p in ["balance", "bal"]):
            balance_idx = i
    
    if debit_idx is None or credit_idx is None:
        return "Multi-line: combine description lines into single row."
    
    def format_aligned_table(rows):
        """Format rows with properly aligned vertical pipes."""
        if not rows:
            return []
        
        num_cols = len(rows[0])
        
        # Calculate max width for each column
        col_widths = [0] * num_cols
        for row in rows:
            for i, val in enumerate(row):
                col_widths[i] = max(col_widths[i], len(val))
        
        # Find last non-empty column index
        last_col = 0
        for row in rows:
            for i, val in enumerate(row):
                if val:
                    last_col = max(last_col, i)
        
        # Ensure empty MIDDLE columns have minimum width
        for i in range(1, last_col):  # Skip first column, only middle columns
            if col_widths[i] == 0:
                col_widths[i] = 7
        
        # Format each row with proper alignment
        formatted = []
        for row in rows:
            # Determine how many columns to include
            end_col = last_col + 2 if last_col < len(row) - 1 else last_col + 1
            end_col = min(end_col, len(row))
            
            # Pad each column value to its width
            parts = []
            for i in range(end_col):
                val = row[i] if i < len(row) else ""
                parts.append(val.ljust(col_widths[i]))
            
            line = " | ".join(parts)
            
            # CRITICAL: If first column is empty, add leading spaces to align pipes
            if not row[0]:
                line = " " * col_widths[0] + " | " + " | ".join(parts[1:])
            
            formatted.append(line)
        
        return formatted
    
    # Create example rows using ACTUAL column positions
    # Credit example (amount in credit_idx position)
    credit_rows = [[""] * num_cols for _ in range(3)]
    credit_rows[0][0] = "a date"
    credit_rows[0][1] = "line 1"
    credit_rows[0][credit_idx] = "85.50"
    # Add Balance column value if it exists
    if balance_idx is not None:
        credit_rows[0][balance_idx] = "$1,085.50 CR"
    credit_rows[1][0] = ""  # Empty date for continuation
    credit_rows[1][1] = "line 2"
    credit_rows[2][0] = "a date"
    credit_rows[2][1] = "line 1 line 2"
    credit_rows[2][credit_idx] = "85.50"
    # Add Balance column value if it exists
    if balance_idx is not None:
        credit_rows[2][balance_idx] = "$1,085.50 CR"
    
    # Debit example (amount in debit_idx position)
    debit_rows = [[""] * num_cols for _ in range(3)]
    debit_rows[0][0] = "a date"
    debit_rows[0][1] = "line 1"
    debit_rows[0][debit_idx] = "150.00"
    # Add Balance column value if it exists
    if balance_idx is not None:
        debit_rows[0][balance_idx] = "$850.00 CR"
    debit_rows[1][0] = ""  # Empty date for continuation
    debit_rows[1][1] = "line 2"
    debit_rows[2][0] = "a date"
    debit_rows[2][1] = "line 1 line 2"
    debit_rows[2][debit_idx] = "150.00"
    # Add Balance column value if it exists
    if balance_idx is not None:
        debit_rows[2][balance_idx] = "$850.00 CR"
    
    # Format both examples
    credit_fmt = format_aligned_table(credit_rows)
    debit_fmt = format_aligned_table(debit_rows)
    
    # Build rule with LABELED examples using actual header names
    rule = f"""  {headers[credit_idx]} example:
       {credit_fmt[0]}
       {credit_fmt[1]}
    you must extract it as:
       {credit_fmt[2]}

  {headers[debit_idx]} example:
       {debit_fmt[0]}
       {debit_fmt[1]}
    you must extract it as:
       {debit_fmt[2]}"""
    
    return rule

# Generate dynamic example rows
example_rows = build_dynamic_example(table_headers, date_col, desc_col, debit_col, credit_col, balance_col)

# Build complete example table with proper alignment
example_table = f"""| {header_string} |
| {separator_row} |
""" + "\n".join([f"| {row} |" for row in example_rows])

# Generate dynamic multi-line rule
multiline_rule = build_multiline_rule(table_headers)

# OPTION D: STRONGER COLUMN REINFORCEMENT
follow_up_prompt = f"""
Extract the transaction table from this bank statement image in markdown format.

Example showing the format I want:

{example_table}

## CRITICAL: COLUMN ALIGNMENT

Before extracting ANY row, locate the header row with these column names:
{" | ".join(table_headers)}

For EACH transaction, you must check which column the amount appears under:

**Step-by-step process:**
1. Find the header row
2. Look at the transaction row
3. Draw an imaginary vertical line from the amount UP to the header
4. Read which header the amount aligns with
5. Put the amount in that SAME column in your markdown table

**Column placement rules:**
- Amount aligns with "{debit_col}" header → put amount in {debit_col} column, leave {credit_col} EMPTY
- Amount aligns with "{credit_col}" header → put amount in {credit_col} column, leave {debit_col} EMPTY

**Do NOT guess based on description text. Use visual alignment ONLY.**

## OTHER RULES

**Multi-line transactions:** Combine description lines into single row:
{multiline_rule}

**Empty columns:** Leave empty (|  |)

**Output:** Markdown table only, no explanations

"""

print("\n" + "=" * 60)
print("Turn 1 Extraction Prompt (Option D):")
print("=" * 60)
print(f"\n{follow_up_prompt}")
print("=" * 60 + "\n")


In [ ]:
# Cell 17.5: Turn 1 - Extract full table (InternVL3-8B version, INDEPENDENT fresh context)

from IPython.display import HTML

def markdown_table_to_html(markdown_text):
    """Convert markdown table to HTML for reliable Jupyter rendering.

    CRITICAL: Markdown tables with empty cells render incorrectly in Jupyter.
    HTML tables preserve empty cells and column alignment properly.
    """
    lines = [line.strip() for line in markdown_text.strip().split('\n') if line.strip()]

    # Find table lines (contain pipes)
    table_lines = [line for line in lines if '|' in line]
    if not table_lines:
        return f"<pre>{markdown_text}</pre>"

    html_parts = ['<table border="1" style="border-collapse: collapse;">']

    for i, line in enumerate(table_lines):
        # Skip separator rows (contain only pipes, hyphens, colons, spaces)
        cleaned = line.replace('|', '').replace('-', '').replace(':', '').replace(' ', '')
        if not cleaned:
            continue

        # Parse cells
        cells = [c.strip() for c in line.split('|')]
        # Remove leading/trailing empty strings from pipe delimiters
        if cells and cells[0] == '':
            cells = cells[1:]
        if cells and cells[-1] == '':
            cells = cells[:-1]

        # First non-separator row is header
        if i == 0:
            html_parts.append('<tr style="background-color: #f0f0f0;">')
            for cell in cells:
                html_parts.append(f'<th style="padding: 8px; text-align: left;">{cell}</th>')
            html_parts.append('</tr>')
        else:
            html_parts.append('<tr>')
            for cell in cells:
                html_parts.append(f'<td style="padding: 8px;">{cell}</td>')
            html_parts.append('</tr>')

    html_parts.append('</table>')
    return '\n'.join(html_parts)

print("🤖 Generating response with InternVL3-8B...")

# CRITICAL: Reload image for fresh context (independent turn, not continuing conversation)
# Uses CONFIG['MAX_TILES'] via load_image() default
pixel_values = load_image(imageName, input_size=448)

# Move to correct device and dtype (MODEL_DTYPE set in cell-5 based on loading mode)
# Vision model is always on GPU 0
pixel_values = pixel_values.to(dtype=MODEL_DTYPE, device='cuda:0')

# Generate response using chat() method with dynamic prompt
cleanedOutput2 = model.chat(
    tokenizer=tokenizer,
    pixel_values=pixel_values,
    question=follow_up_prompt,  # Dynamic prompt with column-aware examples from Cell 19
    generation_config={
        "max_new_tokens": CONFIG['MAX_NEW_TOKENS'],
        "do_sample": False
    }
)

# Clean InternVL3 artifacts:
# 1. Remove image markdown placeholder (![...])
# 2. Remove markdown code fences (```markdown and ```)
lines = cleanedOutput2.split("\n")
cleaned_lines = []
for line in lines:
    stripped = line.strip()
    # Skip image markdown, code fences, and empty fence markers
    if stripped.startswith("!["):
        continue
    if stripped in ["```markdown", "```", "```md"]:
        continue
    cleaned_lines.append(line)

cleanedOutput2 = "\n".join(cleaned_lines)

print("\n✅ Turn 1 extraction complete!")

# Display the extracted table as HTML for reliable rendering with empty cells
print("\n" + "=" * 60)
print("TURN 1 - EXTRACTED TABLE (HTML Rendered):")
print("=" * 60)

# Render as HTML table (fixes empty cell rendering issues with Markdown)
html_table = markdown_table_to_html(cleanedOutput2)
display(HTML(html_table))

print("\n" + "=" * 60)
print("TURN 1 - RAW MARKDOWN TEXT:")
print("=" * 60)
print(cleanedOutput2)
print("=" * 60)

# Save the markdown table
output_path = Path("ivl3_8b_markdown_table_extraction.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput2)

print(f"\n✅ Markdown table saved to: {output_path}")

## Python Parsing and Filtering

Parse the Turn 1 markdown table, filter for debit transactions, and extract schema fields using Python.

In [ ]:
from dateutil import parser as date_parser


def _compute_date_range(dates: list[str]) -> str:
    """
    Compute statement date range ensuring chronological order (oldest - newest).
    
    Bank statements may be in reverse chronological order (newest first),
    so we parse and sort dates to ensure correct order.
    """
    if not dates:
        return "NOT_FOUND"
    
    parsed_dates = []
    for date_str in dates:
        if not date_str or not date_str.strip():
            continue
        try:
            parsed = date_parser.parse(date_str.strip(), dayfirst=True)
            parsed_dates.append((parsed, date_str.strip()))
        except (ValueError, TypeError):
            continue
    
    if not parsed_dates:
        return f"{dates[0]} - {dates[-1]}" if len(dates) >= 2 else dates[0]
    
    parsed_dates.sort(key=lambda x: x[0])
    oldest = parsed_dates[0][1]
    newest = parsed_dates[-1][1]
    
    return f"{oldest} - {newest}"


# Cell 19: Parse Turn 1 markdown table and filter for debits (Python)
import re
from datetime import datetime

def parse_markdown_table(markdown_text):
    """Parse markdown table into list of dictionaries.
    
    CRITICAL: Must preserve empty columns for correct Debit/Credit alignment!
    """
    lines = [line.strip() for line in markdown_text.strip().split('\n') if line.strip()]
    
    # Find header row (first line with pipes)
    header_idx = None
    for i, line in enumerate(lines):
        if '|' in line:
            # Skip separator rows (contain only pipes, hyphens, and spaces)
            cleaned = line.replace('|', '').replace('-', '').replace(' ', '')
            if cleaned:  # Has actual content, not just separators
                header_idx = i
                break
    
    if header_idx is None:
        return []
    
    # Parse headers - KEEP empty values to preserve column positions
    header_line = lines[header_idx]
    header_parts = [h.strip() for h in header_line.split('|')]
    # Remove leading/trailing empty strings from pipe delimiters
    if header_parts and header_parts[0] == '':
        header_parts = header_parts[1:]
    if header_parts and header_parts[-1] == '':
        header_parts = header_parts[:-1]
    # Filter out any remaining empty headers
    headers = [h for h in header_parts if h]
    
    print(f"🔍 Debug: Parsed {len(headers)} headers: {headers}")
    
    # Parse data rows (skip header and separator)
    rows = []
    for idx, line in enumerate(lines[header_idx + 1:], start=header_idx+1):
        if '|' not in line:
            continue
            
        # Skip separator rows
        cleaned = line.replace("|", "").replace("-", "").replace(" ", "").replace(":", "")
        if not cleaned:
            continue
        
        # Parse values - KEEP empty values to preserve column positions!
        value_parts = [v.strip() for v in line.split('|')]
        # Remove leading/trailing empty strings from pipe delimiters
        if value_parts and value_parts[0] == '':
            value_parts = value_parts[1:]
        if value_parts and value_parts[-1] == '':
            value_parts = value_parts[:-1]
        
        print(f"🔍 Debug row {idx}: {len(value_parts)} values: {value_parts}")
        
        # Match to headers length
        if len(value_parts) == len(headers):
            rows.append(dict(zip(headers, value_parts)))
        else:
            print(f"⚠️  Row {idx} mismatch: {len(value_parts)} values vs {len(headers)} headers - SKIPPED")
    
    return rows

def filter_debit_transactions(rows, debit_col):
    """Filter rows to only those with debit (purchase) amounts.
    
    CRITICAL: For tax purposes, we only want transactions where taxpayer PAID money (debits).
    """
    debit_rows = []
    for row in rows:
        debit_value = row.get(debit_col, '').strip()
        # Include row if debit column has a value (not empty)
        if debit_value:
            debit_rows.append(row)
    
    return debit_rows

def extract_schema_fields(rows, date_col, desc_col, debit_col):
    """Extract fields in universal.yaml schema format."""
    if not rows:
        return {
            'TRANSACTION_DATES': 'NOT_FOUND',
            'LINE_ITEM_DESCRIPTIONS': 'NOT_FOUND',
            'TRANSACTION_AMOUNTS_PAID': 'NOT_FOUND',
            'STATEMENT_DATE_RANGE': 'NOT_FOUND'
        }
    
    # Extract lists
    dates = []
    descriptions = []
    amounts = []
    
    for row in rows:
        date = row.get(date_col, '').strip()
        desc = row.get(desc_col, '').strip()
        amount = row.get(debit_col, '').strip()
        
        if date:
            dates.append(date)
        if desc:
            descriptions.append(desc)
        if amount:
            amounts.append(amount)
    
    # Calculate statement date range - use literal date format from image
    # No parsing, no year assumption - just "earliest date - latest date"
    date_range = 'NOT_FOUND'
    if dates:
        # Use first and last date as-is (same format as in the image)
        date_range = _compute_date_range(dates)
    
    return {
        'TRANSACTION_DATES': ' | '.join(dates) if dates else 'NOT_FOUND',
        'LINE_ITEM_DESCRIPTIONS': ' | '.join(descriptions) if descriptions else 'NOT_FOUND',
        'TRANSACTION_AMOUNTS_PAID': ' | '.join(amounts) if amounts else 'NOT_FOUND',
        'STATEMENT_DATE_RANGE': date_range
    }

# Cell: Balance-Based Debit/Credit Validation Function

def detect_document_order(rows, balance_col):
    """
    Detect if document is in chronological or reverse chronological order.
    
    Logic: If balances generally INCREASE going down the table, the document
    is in reverse chronological order (newest first, oldest last).
    
    Returns:
        str: "chronological" or "reverse_chronological"
    """
    def parse_amount(value):
        if not value or value.strip() == "":
            return 0.0
        cleaned = value.replace("$", "").replace(",", "").replace("CR", "").replace("DR", "").strip()
        try:
            return float(cleaned)
        except ValueError:
            return 0.0
    
    if len(rows) < 3:
        return "chronological"  # Default assumption
    
    # Sample multiple balance transitions to determine trend
    increases = 0
    decreases = 0
    
    for i in range(1, min(len(rows), 10)):  # Check first 10 rows
        prev_balance = parse_amount(rows[i-1].get(balance_col, "0"))
        curr_balance = parse_amount(rows[i].get(balance_col, "0"))
        
        if curr_balance > prev_balance + 0.01:
            increases += 1
        elif curr_balance < prev_balance - 0.01:
            decreases += 1
    
    # If balances mostly increase going down → reverse chronological
    if increases > decreases:
        return "reverse_chronological"
    else:
        return "chronological"


def validate_and_correct_alignment(rows, balance_col, debit_col, credit_col, desc_col):
    """
    Use balance changes to validate and correct debit/credit alignment.
    
    CRITICAL: This provides mathematical proof of correct alignment!
    
    IMPORTANT: This function now detects document order (chronological vs 
    reverse chronological) and adjusts validation logic accordingly.
    
    For CHRONOLOGICAL order (oldest first):
    - Balance increase between rows = CREDIT (income/deposit)
    - Balance decrease between rows = DEBIT (expense/withdrawal)
    
    For REVERSE CHRONOLOGICAL order (newest first, like image_008.png):
    - Balance increase between rows = DEBIT (going back in time, balance was lower before)
    - Balance decrease between rows = CREDIT (going back in time, balance was higher before)
    """
    
    def parse_amount(value):
        """Extract numeric value from formatted currency string."""
        if not value or value.strip() == "":
            return 0.0
        cleaned = value.replace("$", "").replace(",", "").replace("CR", "").replace("DR", "").strip()
        try:
            return float(cleaned)
        except ValueError:
            return 0.0
    
    def is_balance_row(row, desc_col):
        """Check if this row is an opening/closing balance row (not a transaction)."""
        desc = row.get(desc_col, "").upper()
        return "OPENING BALANCE" in desc or "CLOSING BALANCE" in desc
    
    # Check if Balance column exists
    if not rows or balance_col not in rows[0]:
        print(f"⚠️  Balance column '{balance_col}' not found - skipping validation")
        return rows
    
    # CRITICAL: Detect document order before validation
    doc_order = detect_document_order(rows, balance_col)
    print(f"📊 Document order detected: {doc_order.upper()}")
    
    if doc_order == "reverse_chronological":
        print("💡 Reverse chronological: Balance increases going down = transactions went BACK in time")
        print("💡 Validation logic inverted: balance increase → DEBIT, balance decrease → CREDIT")
    
    corrected_rows = []
    corrections_made = 0
    start_idx = 0
    
    # Check if row 0 is an opening balance row
    if rows and is_balance_row(rows[0], desc_col):
        print(f"✅ Row 0: Opening/closing balance detected - skipping (not a transaction)")
        start_idx = 1
    elif rows:
        corrected_rows.append(rows[0].copy())
        print(f"✅ Row 0: First transaction included without validation (no previous balance to compare)")
        start_idx = 1
    
    # Validate and correct remaining rows
    for i in range(start_idx, len(rows)):
        current_row = rows[i].copy()
        
        if is_balance_row(current_row, desc_col):
            print(f"⚠️  Row {i}: Opening/closing balance row detected - skipping")
            continue
        
        prev_idx = i - 1
        while prev_idx >= 0 and is_balance_row(rows[prev_idx], desc_col):
            prev_idx -= 1
        
        if prev_idx < 0:
            corrected_rows.append(current_row)
            print(f"✅ Row {i}: First transaction after balance row - included without validation")
            continue
        
        prev_balance = parse_amount(rows[prev_idx].get(balance_col, "0"))
        curr_balance = parse_amount(current_row.get(balance_col, "0"))
        balance_change = curr_balance - prev_balance
        
        debit_value = parse_amount(current_row.get(debit_col, ""))
        credit_value = parse_amount(current_row.get(credit_col, ""))
        
        # CRITICAL: Adjust validation logic based on document order
        if doc_order == "reverse_chronological":
            if balance_change > 0.01:  # Balance increased (reverse chrono = DEBIT)
                if credit_value > 0 and debit_value == 0:
                    print(f"⚠️  Row {i}: Balance increased by ${balance_change:.2f} (reverse chrono → DEBIT expected) but amount in Credit column")
                    print(f"   Correction: Moving ${credit_value:.2f} from Credit → Debit")
                    current_row[debit_col] = current_row[credit_col]
                    current_row[credit_col] = ""
                    corrections_made += 1
            elif balance_change < -0.01:  # Balance decreased (reverse chrono = CREDIT)
                if debit_value > 0 and credit_value == 0:
                    print(f"⚠️  Row {i}: Balance decreased by ${abs(balance_change):.2f} (reverse chrono → CREDIT expected) but amount in Debit column")
                    print(f"   Correction: Moving ${debit_value:.2f} from Debit → Credit")
                    current_row[credit_col] = current_row[debit_col]
                    current_row[debit_col] = ""
                    corrections_made += 1
        else:
            if balance_change > 0.01:  # Balance increased = CREDIT
                if debit_value > 0 and credit_value == 0:
                    print(f"⚠️  Row {i}: Balance increased by ${balance_change:.2f} but amount in Debit column")
                    print(f"   Correction: Moving ${debit_value:.2f} from Debit → Credit")
                    current_row[credit_col] = current_row[debit_col]
                    current_row[debit_col] = ""
                    corrections_made += 1
            elif balance_change < -0.01:  # Balance decreased = DEBIT
                if credit_value > 0 and debit_value == 0:
                    print(f"⚠️  Row {i}: Balance decreased by ${abs(balance_change):.2f} but amount in Credit column")
                    print(f"   Correction: Moving ${credit_value:.2f} from Credit → Debit")
                    current_row[debit_col] = current_row[credit_col]
                    current_row[credit_col] = ""
                    corrections_made += 1
        
        corrected_rows.append(current_row)
    
    print(f"\n✅ Balance validation complete: {corrections_made} corrections made")
    print(f"✅ Total transaction rows processed: {len(corrected_rows)}")
    return corrected_rows

print("✅ Balance validation function defined")
print("💡 This function uses balance mathematics to validate and auto-correct misaligned amounts")
print("💡 Opening/closing balance rows are automatically detected and skipped")
print("💡 Automatically detects reverse chronological documents and adjusts logic")


print("=" * 60)
print("PARSING TURN 1 MARKDOWN TABLE:")
print("=" * 60)

# Parse the full markdown table from Turn 1
all_rows = parse_markdown_table(cleanedOutput2)

print(f"\n📊 Parsed {len(all_rows)} total transactions from Turn 1 markdown table")

if all_rows:
    # Show sample parsed row
    print(f"\n🔍 Sample parsed row:")
    for key, value in all_rows[0].items():
        print(f"  {key}: '{value}'")

# Filter to only debit (purchase) transactions - Python filtering, not LLM!

# Apply balance-based validation if Balance column exists
if balance_col and balance_col in table_headers:
    print("\n" + "=" * 60)
    print("BALANCE-BASED VALIDATION:")
    print("=" * 60)
    all_rows = validate_and_correct_alignment(all_rows, balance_col, debit_col, credit_col, desc_col)
else:
    print("\n⚠️  No Balance column - skipping balance-based validation")

debit_rows = filter_debit_transactions(all_rows, debit_col)

print(f"\n💰 Filtered to {len(debit_rows)} debit transactions (taxpayer purchases)")
print("\n" + "=" * 60)
print("DEBIT TRANSACTIONS (WHAT TAXPAYER PAID):")
print("=" * 60)
for i, row in enumerate(debit_rows, 1):
    print(f"\nTransaction {i}:")
    print(f"  {date_col}: {row.get(date_col, '')}")
    print(f"  {desc_col}: {row.get(desc_col, '')}")
    print(f"  {debit_col}: {row.get(debit_col, '')}")

# Extract schema fields using the LITERAL column names from pattern matching
schema_fields = extract_schema_fields(debit_rows, date_col, desc_col, debit_col)

print("\n" + "=" * 60)
print("EXTRACTED SCHEMA FIELDS (TAX-RELEVANT DATA):")
print("=" * 60)
for field, value in schema_fields.items():
    print(f"{field}: {value}")
print("=" * 60)

# Save to file with ivl3_8b prefix
output_path = Path("ivl3_8b_extracted_fields.txt")
with output_path.open("w", encoding="utf-8") as f:
    for field, value in schema_fields.items():
        f.write(f"{field}: {value}\n")

print(f"\n✅ Schema fields saved to: {output_path}")
print(f"💡 Fields extracted from columns: '{date_col}' | '{desc_col}' | '{debit_col}'")
print(f"🎯 Success: Python parsing + filtering from Turn 1 markdown table")